In [1]:
import sys
import requests
import json
import pandas as pd
from datetime import datetime, timedelta

from pprint import PrettyPrinter
pp = PrettyPrinter(indent=4)



In [2]:
def adjust_sensors_delay_time(implementation):
    """
    This function returns the delay time for each sensor in order to adjust the data
    """
    #esp32
    seconds_delay = 0
    milliseconds_delay = 0
    if implementation == "ESP32S3": 
        seconds_delay = 2
        milliseconds_delay = 920

    return timedelta(seconds=seconds_delay, milliseconds=milliseconds_delay)

In [3]:
def jobs_unix_timestamp(implementation, day, month):
    epoch = datetime.utcfromtimestamp(0)

    #startDT = datetime(2023, month, day, 9, 0, 0, 0) - timedelta(hours=1) + adjust_sensors_delay_time(implementation)
    startDT = datetime(2023, month, day, 9, 0, 0, 0) - timedelta(hours=1) + adjust_sensors_delay_time(implementation)
    startTS = int((startDT - epoch).total_seconds() * 1000.0)
    #endDT = datetime(2023, month, day, 18, 0, 0 , 0) - timedelta(hours=1) + adjust_sensors_delay_time(implementation)
    endDT = datetime(2023, month, day, 18, 0, 0 , 0) - timedelta(hours=1) + adjust_sensors_delay_time(implementation)
    endTS = int((endDT - epoch).total_seconds() * 1000.0)
    

    return str(startTS), str(endTS)

In [4]:
RFID_analysis_keys = [
    'gabarit'
]

TASK_analysis_keys = [
    'task', 'empid', 'sec'
]
TASKTOTAL_analysis_keys = [
    'emp_id', 'taskid','totalm', 'av','pr' 
]
BLE_analysis_keys = [
    'id','rssi'
]


BLE_analysis_keys = [
    'id','rssi'
]


In [5]:
def json_csv(data, dataset, implementation):
    data_df = pd.DataFrame()
    first_flag = True
    
    if dataset == "rfid":
        analysis_keys = RFID_analysis_keys
    elif dataset == "task":
        analysis_keys = TASK_analysis_keys
    else:
        analysis_keys = BLE_analysis_keys



    for analysis_key in analysis_keys:
        if first_flag:
            data_df = pd.DataFrame(data[analysis_key])
            data_df = data_df.rename(columns={'value': analysis_key})
            first_flag = False
        else:
            data_df = data_df.join(pd.DataFrame(data[analysis_key]).set_index('ts'), on='ts')
            data_df = data_df.rename(columns={'value': analysis_key})
    # change the timestamp to seconds
    data_df['ts'] = pd.to_datetime(data_df['ts']/1000,unit='s') + timedelta(hours=1) - adjust_sensors_delay_time(implementation)
    data_df['ts']= data_df['ts'].astype('datetime64[s]')
    return data_df.iloc[::-1]

def getToken():
    username = 'isabel.portugal@tecnico.ulisboa.pt' #email used to create your ThingsBoard account (your email associated with tennant)
    password = 'tHn%Mv1pnmq2av='		#password of your ThingsBoard account (or your tennant account)
    url = 'http://iot.istartlab.tecnico.ulisboa.pt/api/auth/login'
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    loginJSON = {'username': username, 'password': password}
    tokenAuthResp = requests.post(url, headers=headers, json=loginJSON).json()
    #print(tokenAuthResp)
    token = tokenAuthResp['token']
    return token

In [9]:
tkn = getToken()

my_headers = {'X-Authorization':  "Bearer " + tkn, "Content-Type" : "application/json"}
#print(my_headers)
#url to test connection
url = "http://iot.istartlab.tecnico.ulisboa.pt/api/plugins/telemetry/DEVICE/27110c40-2a6b-11ee-9b43-67453a5ac630/keys/timeseries"

response = requests.get(url, headers=my_headers)
print(response) , print(response.text)

my_headers = {'X-Authorization':  "Bearer " + tkn, "Content-Type" : "application/json"}
#print(my_headers)

<Response [404]>
Device with requested id wasn't found!


In [11]:
# CHANGE ONLY THIS
month_int = 10
day_int = 11
#dataset = "rfid" #"task"
dataset = "rfid"

day_str = str(day_int).zfill(2)
month_str = str(month_int).zfill(2)

startTS, endTS = jobs_unix_timestamp("ESP32S3", day_int, month_int)

limit = "1000000000"

date_filename = dataset + "_" + day_str + "_" + month_str

cnt = 1

if dataset == "rfid":
    devices = ["27110c40-2a6b-11ee-9b43-67453a5ac633","f4765800-2cc5-11ee-a22d-b5d6c11ab119","10fc9700-2cc6-11ee-a22d-b5d6c11ab119"]
    #for analysis_key in RFID_analysis_keys:
    keys = "ts,gabarit"

elif dataset == "task":
    devices = ["502ba360-d205-11ed-a394-2dc30865e203"]
    #for analysis_key in TASK_analysis_keys:
    keys = "task,empid,sec" 

else:
    devices = ["8516caf0-7a71-11ed-a394-2dc30865e203","fe7d6ff0-b52e-11ed-a394-2dc30865e203","4b593330-2498-11ee-a394-2dc30865e203" ]
    #1: fim da linha, 2: pacotes, 3: elevator
    #for analysis_key in BLE_analysis_keys:
    keys = "id,rssi"   

print(keys)

for dev in devices:
    url = "http://iot.istartlab.tecnico.ulisboa.pt/api/plugins/telemetry/DEVICE/" + dev + "/values/timeseries?keys=" + keys + "&startTs=" + startTS + "&endTs=" + endTS + "&limit=" + limit
    response = requests.get(url, headers=my_headers)
    print("response received", response.text)
    data = json.loads(response.text)
    data_df = json_csv(data, dataset, "ESP32S3")
    if dataset == "rfid" or dataset == "ble":
        data_df.to_csv(date_filename + "_" + str(cnt) + ".csv")
    else:
        data_df.to_csv(date_filename + "_" + ".csv")
                   #
    cnt += 1

print("acc done")


ts,gabarit
response received {"gabarit":[{"ts":1697041426937,"value":"0"},{"ts":1697041421258,"value":"21"},{"ts":1697041297378,"value":"0"},{"ts":1697041292138,"value":"15"},{"ts":1697041284765,"value":"0"},{"ts":1697041119777,"value":"0"},{"ts":1697041114046,"value":"6"},{"ts":1697041006435,"value":"0"},{"ts":1697040998677,"value":"5"},{"ts":1697040952569,"value":"0"},{"ts":1697040952568,"value":"8"},{"ts":1697040770565,"value":"0"},{"ts":1697040766089,"value":"2"},{"ts":1697040600437,"value":"0"},{"ts":1697040590357,"value":"11"},{"ts":1697040587677,"value":"11"},{"ts":1697040580292,"value":"0"},{"ts":1697040548646,"value":"24"},{"ts":1697040481837,"value":"0"},{"ts":1697040475857,"value":"18"},{"ts":1697040473777,"value":"18"},{"ts":1697040471745,"value":"18"},{"ts":1697040469697,"value":"18"},{"ts":1697040467640,"value":"18"},{"ts":1697040465638,"value":"18"},{"ts":1697040463558,"value":"18"},{"ts":1697040461509,"value":"18"},{"ts":1697040459479,"value":"18"},{"ts":1697040457438,"

In [8]:
print(response.text)

{"gabarit":[{"ts":1696870765494,"value":"0"},{"ts":1696870760441,"value":"13"},{"ts":1696870757302,"value":"0"},{"ts":1696870752309,"value":"13"},{"ts":1696870720516,"value":"0"},{"ts":1696870715501,"value":"13"},{"ts":1696870701681,"value":"0"},{"ts":1696870696680,"value":"5"},{"ts":1696870692869,"value":"5"},{"ts":1696870686341,"value":"0"},{"ts":1696870681329,"value":"5"},{"ts":1696870650884,"value":"0"},{"ts":1696870645669,"value":"5"},{"ts":1696870556689,"value":"0"},{"ts":1696870551669,"value":"1"},{"ts":1696870549669,"value":"1"},{"ts":1696870547669,"value":"1"},{"ts":1696870545661,"value":"1"},{"ts":1696870543641,"value":"1"},{"ts":1696870541640,"value":"1"},{"ts":1696870539609,"value":"1"},{"ts":1696870537360,"value":"1"},{"ts":1696870523100,"value":"0"},{"ts":1696870518080,"value":"1"},{"ts":1696870516061,"value":"1"},{"ts":1696870510860,"value":"0"},{"ts":1696870505829,"value":"1"},{"ts":1696870314481,"value":"0"},{"ts":1696870309460,"value":"10"},{"ts":1696870306867,"value"